Machine learning is a subset of artificial intelligence that enables computers to learn from data without being explicitly programmed. It involves training algorithms on large datasets to recognize patterns, make predictions, and improve their performance over time. Machine learning models can be trained on various types of data, such as images, text, or audio, and can be applied to a wide range of applications, including natural language processing, computer vision, and predictive analytics. Through machine learning, computers can learn to identify objects in images, understand human language, and make predictions about future events, all without being explicitly programmed to do so.

## Setup

In [6]:
from pathlib import Path
import shutil
import glob
from opensoundscape import Audio
from opensoundscape.spectrogram import MelSpectrogram, Spectrogram
import os


src_data_dir = 'data/audio/training_sets/small'
test_data_dir = 'data/audio/training_sets/test_data'
train_data_dir = 'data/audio/training_sets/train_data'
temp_dir = 'data/audio/temp'
model_dir = 'data/models'
model_file = os.path.join(model_dir, "model.keras")

# data gen
num_train_imgs = 10
num_val_imgs = 250

# bandpass filters
high_cut = 4000
low_cut = 300

# sounds
clip_seconds = 1
overlap_seconds = .1
freq = 44100

image_shape = (200, 200)

classes = list()
for path in Path(src_data_dir).iterdir():
    if path.is_dir():
        classes.append(path.name)

num_classes = len(classes)

# Data Aqcusition

For this portion, you may either use the training set included in the supplementary audio files, or generate a dataset from iNaturalist. By default the script assumes you are working with the supplementary dataset. 

In [7]:
supplementary = True

# get data from iNaturalist
if not supplementary:
    pass

## Data Processing / Spectrogram Creation

In [8]:
# ETL process for creating spectrogram images from .wav files

# delete existing training directories
for species in classes:
    if os.path.exists(os.path.join(train_data_dir, species)):
        shutil.rmtree(os.path.join(train_data_dir, species))

    # make directories for each species
    os.makedirs(os.path.join(train_data_dir, species))

    for wav in [file for file in glob.glob(os.path.join(src_data_dir, species, "*.wav"))]:
        audio_object = Audio.from_file(wav)
        audio_object = audio_object.resample(freq)
        audio_object = audio_object.bandpass(low_f=low_cut, high_f=high_cut, order= 12)
        
        clips, clip_df = audio_object.split(clip_duration=clip_seconds, clip_overlap=overlap_seconds, final_clip=None)
        spectrogram_objects = [Spectrogram.from_audio(audio_object, window_samples=400).bandpass(min_f=low_cut, max_f=high_cut) for audio_object in clips]

        count = 0
        for spectrogram_object in spectrogram_objects:
            spectrogram_image = spectrogram_object.to_image(shape=image_shape)
            spectrogram_image.save(os.path.join(train_data_dir,species, count.__str__() + ".png"))
            count += 1

In [9]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [10]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

batch_size = 32

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    train_data_dir,
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_shape,
    batch_size=batch_size,
    label_mode="categorical",
    labels="inferred",
    class_names=classes,
)

train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)


def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.25)(x)

    if num_classes == 2:
        units = 1
        outputs = layers.Dense(units, activation="sigmoid")(x)
    else:
        units = num_classes
        outputs = layers.Dense(units, activation="softmax")(x)

    return keras.Model(inputs, outputs)

model = make_model(input_shape=image_shape + (3,), num_classes=num_classes)

epochs = 25

callbacks = [
    # keras.callbacks.ModelCheckpoint(model_dir + "\save_at_{epoch}.keras"),
    keras.callbacks.ModelCheckpoint(model_file),
]
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

Found 649 files belonging to 12 classes.
Using 520 files for training.
Using 129 files for validation.
Epoch 1/25


2024-09-26 13:47:17.740340: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:524] Loaded runtime CuDNN library: 8.5.0 but source was compiled with: 8.9.6.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2024-09-26 13:47:17.931320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:524] Loaded runtime CuDNN library: 8.5.0 but source was compiled with: 8.9.6.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2024-09-26 13:47:17.944138: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at xla_ops.cc:577 

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/john/Desktop/ml-playground/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/john/Desktop/ml-playground/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/john/Desktop/ml-playground/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/john/Desktop/ml-playground/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/home/john/Desktop/ml-play

In [ ]:
# clean up iNaturalist data to save disk space

# TODO